In [1]:
import pandas as pd
import numpy as np
import nltk

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

from gensim import utils
import re

from gensim.models.doc2vec import LabeledSentence
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score

In [2]:
#Import Data
df=pd.read_csv('./questions.csv')

In [3]:
df.head(10)

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
6,6,13,14,Should I buy tiago?,What keeps childern active and far from phone ...,0
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
8,8,17,18,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0
9,9,19,20,Motorola (company): Can I hack my Charter Moto...,How do I hack Motorola DCX3400 for free internet?,0


In [4]:
# Check for null values
df[df.isnull().any(axis=1)]

# Drop rows with null Values
df.drop(df[df.isnull().any(axis=1)].index,inplace=True)

In [5]:
def review_to_wordlist(review, remove_stopwords=True):
    # Clean the text, with the option to remove stopwords.
    
    # Convert words to lower case and split them
    words = review.lower().split()

    # Optionally remove stop words (true by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    
    review_text = " ".join(words)

    # Clean the text
    review_text = re.sub(r"[^A-Za-z0-9(),!.?\'\`]", " ", review_text)
    review_text = re.sub(r"\'s", " 's ", review_text)
    review_text = re.sub(r"\'ve", " 've ", review_text)
    review_text = re.sub(r"n\'t", " 't ", review_text)
    review_text = re.sub(r"\'re", " 're ", review_text)
    review_text = re.sub(r"\'d", " 'd ", review_text)
    review_text = re.sub(r"\'ll", " 'll ", review_text)
    review_text = re.sub(r",", " ", review_text)
    review_text = re.sub(r"\.", " ", review_text)
    review_text = re.sub(r"!", " ", review_text)
    review_text = re.sub(r"\(", " ( ", review_text)
    review_text = re.sub(r"\)", " ) ", review_text)
    review_text = re.sub(r"\?", " ", review_text)
    review_text = re.sub(r"\s{2,}", " ", review_text)
    
    words = review_text.split()
    
    # Shorten words to their stems
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in words]
    
    review_text = " ".join(stemmed_words)
    
    # Return a list of words
    return(review_text)

In [6]:
def process_questions(question_list, questions, question_list_name):
    for question in questions:
        question_list.append(review_to_wordlist(question))
        if len(question_list) % 10000 == 0:
            progress = len(question_list)/len(df) * 100
            print("{} is {}% complete".format(question_list_name, round(progress, 1)))
            

In [7]:
questions1 = []
process_questions(questions1, df.question1, "questions1")

questions2 = []
process_questions(questions2, df.question2, "questions2")

questions1 is 2.5% complete
questions1 is 4.9% complete
questions1 is 7.4% complete
questions1 is 9.9% complete
questions1 is 12.4% complete
questions1 is 14.8% complete
questions1 is 17.3% complete
questions1 is 19.8% complete
questions1 is 22.3% complete
questions1 is 24.7% complete
questions1 is 27.2% complete
questions1 is 29.7% complete
questions1 is 32.2% complete
questions1 is 34.6% complete
questions1 is 37.1% complete
questions1 is 39.6% complete
questions1 is 42.0% complete
questions1 is 44.5% complete
questions1 is 47.0% complete
questions1 is 49.5% complete
questions1 is 51.9% complete
questions1 is 54.4% complete
questions1 is 56.9% complete
questions1 is 59.4% complete
questions1 is 61.8% complete
questions1 is 64.3% complete
questions1 is 66.8% complete
questions1 is 69.2% complete
questions1 is 71.7% complete
questions1 is 74.2% complete
questions1 is 76.7% complete
questions1 is 79.1% complete
questions1 is 81.6% complete
questions1 is 84.1% complete
questions1 is 86.6

In [8]:
questions_labeled = []

for i in range(len(questions1)):
    # Question strings need to be separated into words
    # Each question needs a unique label
    questions_labeled.append(TaggedDocument(questions1[i].split(), df[df.index == i].qid1))
    questions_labeled.append(TaggedDocument(questions2[i].split(), df[df.index == i].qid2))
    if i % 1000 == 0:
        progress = i/len(questions1) * 100
        print("{}% complete".format(round(progress, 2)))

0.0% complete
0.25% complete
0.49% complete
0.74% complete
0.99% complete
1.24% complete
1.48% complete
1.73% complete
1.98% complete
2.23% complete
2.47% complete
2.72% complete
2.97% complete
3.22% complete
3.46% complete
3.71% complete
3.96% complete
4.2% complete
4.45% complete
4.7% complete
4.95% complete
5.19% complete
5.44% complete
5.69% complete
5.94% complete
6.18% complete
6.43% complete
6.68% complete
6.92% complete
7.17% complete
7.42% complete
7.67% complete
7.91% complete
8.16% complete
8.41% complete
8.66% complete
8.9% complete
9.15% complete
9.4% complete
9.65% complete
9.89% complete
10.14% complete
10.39% complete
10.63% complete
10.88% complete
11.13% complete
11.38% complete
11.62% complete
11.87% complete
12.12% complete
12.37% complete
12.61% complete
12.86% complete
13.11% complete
13.35% complete
13.6% complete
13.85% complete
14.1% complete
14.34% complete
14.59% complete
14.84% complete
15.09% complete
15.33% complete
15.58% complete
15.83% complete
16.08% c

In [9]:
model = Doc2Vec(dm = 1, min_count=1, window=10, vector_size=150, sample=1e-4, negative=10)

C:\Users\jaisu\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\models\base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "


In [10]:
model.build_vocab(questions_labeled)

In [14]:
#model.train(questions_labeled, epochs=5, total_examples=model.corpus_count)
Skipping over the training portion of the code

In [15]:
model.wv.most_similar("book")

[('read', 0.758697509765625),
 ('novel', 0.7352903485298157),
 ('newspap', 0.6765315532684326),
 ('author', 0.6548153162002563),
 ('ncert', 0.6481190919876099),
 ('textbook', 0.6436299681663513),
 ('publish', 0.6311969757080078),
 ('magazin', 0.6250003576278687),
 ('fiction', 0.6204158067703247),
 ('recommend', 0.613125205039978)]

In [16]:
from nltk.corpus import wordnet as wn